# 1 Break问题

动态规划中有一类问题是对给定的字符串或数字进行**Break**求解问题。例如[343号题Integer Break](https://leetcode.com/problems/integer-break/)，以及[279号题Perfect Squares](https://leetcode.com/problems/perfect-squares/)。这类题目一般都是由多个最优子结构所组成，即通过对原问题Break后，可以对子问题进行求解，子问题求解后即可得到原问题的解，一般需要使用两重循环DP来进行解决。

# 2 Integer Break问题

问题描述：给定一个正整数$n$，将它至少分成两个正整数$a$和$b$，且$a+b=n$，使得$a\times b$的积最大，返回最大的积。

这个问题可以先通过**分解子问题**来进行求解，以$n=4$为例：

![](images/integer-break.png)

当我们需要得到分割$n=4$的解时，可以将问题变为：
- 1.分解为$1+3$，然后求解分割3的解
- 2.分解为$2+2$，然后求解分割2的解
- 3.分解为$3+1$，然后求解分割1的解

如上图所示，我们可以发现只要我们能够解决子问题，就可以通过对上面3个子问题的结果进行比较，得到最终的解。进一步地，就可以很容易地联想利用递归去进行求解。考虑到在上图中，我们可以发现有很多**重叠子问题**，因此采用“递归+记忆化搜索来进行求解”。

我们定义函数$f(i)$代表分割数字$i$所能得到的最大乘积（且$i$至少要被分为两份），那么最终问题就变为：$\max\{f(i-1)\times 1, f(i-2)\times 2, \cdots, f(1) \times (i-1)\}$。上述的公式之间少考虑了$(i-j)\times j$这一项，我们回顾定义，$f(i)$是指分割数字$i$至少为两份之后能够得到的最大乘积，因此$f(i-j)\times j$已经使得数字$i$被分成了3份。

因此综合上述分析，我们可以得到：

$$\max\{f(i-j)\times j, (i-j)\times j\}, where 1\le j\le i-1$$

## 2.1 记忆化搜索解法

代码如下：


```java
class Solution {
    int[] memo;
    public int integerBreak(int n) {
        // 思路：递归 + 记忆化搜索
        
        memo = new int[n+1];
        Arrays.fill(memo, -1);
        return helper(n);
    }
    
    private int helper(int n) {
        // 递归结束条件
        if( n == 1 )
            return 1;
        
        // 递归过程
        if( memo[n] != -1 )
            return memo[n];

        for( int i = 1; i < n; i++ ) {
            memo[n] = Math.max( memo[n], Math.max( i * (n-i), helper(i) * (n-i) ) );
        }
        
        return memo[n];
    }
}
```

## 2.2 动态规划方法

从上述的分析中，我们可以很容易地定义出动态规划的状态和状态转移方程：

- 状态定义：$f(i)$代表分割数字$i$至少为两份后所能得到的最大乘积
- 状态转移：$f(i)=\max\{f(i-j) \times j, (i-j)\times j \}, where 1\le j \le i-1$

代码如下：

```java
class Solution {
    int[] memo;
    public int integerBreak(int n) {
        // 动态规划
        // 时间复杂度：O(n2)
        // 空间复杂度：O(n)
        assert( n >= 2 );
        
        int[] dp = new int[n+1];
        dp[1] = 1;
        
        for( int i = 2; i <= n; i++ ) {
            for( int j = 1; j <= i-1; j++ ) {
                dp[i] = Math.max( dp[i], Math.max( (i-j) * j, dp[i-j] * j ) );
            }
        }
        
        return dp[n];
    }
}
```

# 3 Perfect Square问题

问题描述：给定一个正整数$n$，寻找最小的完全平方数个数，使得这些完全平方数和为$n$。例如$4 = 1+1+1+1$或$4=4$，此时$4$本身就是一个完全平方数，直接返回1。



这个问题也是一个**Break**问题，Integer Break是通过break数字后求积，而Perfect Square问题是通过break数字为完全平方数后，求最小个数。我们同样沿袭Integer Break问题的思路来进行思考，对于数字$i$，我们定义它可以被完全平方数表示的方式有：

- $1+f(i-1)$
- $4+f(i-4)$
- $9+f(i-9)$
- $\cdots$

则我们可以定义状态和状态转移：

- 状态函数：$f(i)$代表数字$i$可以所需的最小完全平方数个数；
- 状态转移：$f(i) = \min\{f(i-1), f(i-4), \cdots, \}+1$

> 问题：对于给定任意正整数$n$，该问题一定有解吗？答：一定有解，因为$1$是完全平方数，因此$n$可以表示为$n$个$1$。

代码如下：

```java
class Solution {
    public int numSquares(int n) {
        // 动态规划
        // 时间复杂度：O(n2)
        // 空间复杂度：O(n)
        
        assert( n > 0 );
        
        // 初始化数组
        int[] dp = new int[n+1];
        
        // DP求解
        for( int i = 1; i <= n; i++ ) {
            dp[i] = i;
            for( int j = 1; i-j*j >= 0; j++ ) {
                dp[i] = Math.min(dp[i], dp[i-j*j] + 1);
            }
        }
        
        return dp[n];
    }
}
```

# 4 总结

上述Break类的问题，都是具有重叠子问题和最优子结构的，因此思路上都是以求解子问题为主。